# Infinite Families for the Godsil-Lato Graph on 8 Vertices

Is there an infinite family that generalizes the Godsil-Lato graph on 8 vertices, whose adjacency matrix is $I_2 \otimes H(C_4) + \sigma_y \otimes J_4$?

In [1]:
# %load adj_functions.sage
import numpy as np
from numpy import linalg

################################################################################
# Function: idempotents                                                        #
#                                                                              #
# Purpose: find principal idempotents of a Hermitian matrix                    #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#                                                                              #
# Returns: principal idempotents, eigenvalues                                  #
#                                                                              #
################################################################################

def idempotents(A):
    if not matrix(A).is_hermitian():
        print("A is not Hermitian")
        return []
    # Extracts eigenvalues and eigenvectors
    W,V = linalg.eigh(A)
    # Converts eigenvectors into orthonormal basis
    Q, R = linalg.qr(V) 
    Q = matrix(Q)
    # Number of eigenvectors
    A = np.matrix(A)
    n = len(A)
    idems = []
    evals = []
    i = 0
    j = 0
    while i < n:
        # Zero matrix
        E = matrix(n, n, 0)
        # Checks if eigenvectors share an eigenvalue
        while j < n and abs(W[i] - W[j]) < 0.01:
            # Converts eigenvector into column vecror
            v = Q.column(j).column()
            # updates idempotent
            E = E + v*v.conjugate_transpose();
            j = j + 1
        E = E
        idems.append(E)
        evals.append(W[i].round(3))
        i = j
    return idems, evals

################################################################################
# Function: transfer                                                           #
#                                                                              #
# Purpose: return transfer matrix U(t) given adjacency matrix/Hamiltonian      #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   t        float                              time                           #
#   k        integer                            rounding accuracy              #
#                                                                              #
# Returns: transfer U(t)                                                       #
#                                                                              #
################################################################################

def transfer(A, t):
    if not matrix(A).is_hermitian():
        print("A is not Hermitian")
        return []
    idems, evals = idempotents(A)
    # Number of eigenvectors
    A = np.matrix(A)
    n = len(A)
    # Zero matrix
    U = matrix(n, n, 0)
    for i in range(len(idems)):
        # Updates transder matrix
        U = U + exp(-I*t*evals[i])*idems[i]
    return U

# Base case

In [2]:
I2 = matrix([[1,0],[0,1]])
HC4 = np.matrix([[0,-1j,0,1j],[1j,0,-1j,0],[0,1j,0,-1j],[-1j,0,1j,0]]); HC4 = matrix(HC4)
SY = np.matrix([[0,-1j],[1j,0]]); SY = matrix(SY)
J4 = matrix([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]])
A2 =  I2.tensor_product(HC4) + SY.tensor_product(J4); A2

[   0.0 -1.0*I    0.0  1.0*I -1.0*I -1.0*I -1.0*I -1.0*I]
[ 1.0*I    0.0 -1.0*I    0.0 -1.0*I -1.0*I -1.0*I -1.0*I]
[   0.0  1.0*I    0.0 -1.0*I -1.0*I -1.0*I -1.0*I -1.0*I]
[-1.0*I    0.0  1.0*I    0.0 -1.0*I -1.0*I -1.0*I -1.0*I]
[ 1.0*I  1.0*I  1.0*I  1.0*I    0.0 -1.0*I    0.0  1.0*I]
[ 1.0*I  1.0*I  1.0*I  1.0*I  1.0*I    0.0 -1.0*I    0.0]
[ 1.0*I  1.0*I  1.0*I  1.0*I    0.0  1.0*I    0.0 -1.0*I]
[ 1.0*I  1.0*I  1.0*I  1.0*I -1.0*I    0.0  1.0*I    0.0]

In [3]:
transfer(A2, np.pi/2).round(5)

[0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0]
[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0]
[0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0]

In [4]:
E, eVal = idempotents(A2)

for i in range(len(eVal)):
    print("Idempotent for", int(eVal[i]))
    print(E[i].round(5))

Idempotent for -4
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
Idempotent for -2
[   0.25  0.25*I   -0.25 -0.25*I     0.0     0.0     0.0     0.0]
[-0.25*I    0.25  0.25*I   -0.25     0.0     0.0     0.0     0.0]
[  -0.25 -0.25*I    0.25  0.25*I     0.0     0.0     0.0     0.0]
[ 0.25*I   -0.25 -0.25*I    0.25     0.0     0.0     0.0     0.0]
[    0.0     0.0     0.0     0.0    0.25  0.25*I   -0.25 -0.25*I]
[    0.0     0.0     0.0     0.0 -0.25*I  

# Case which gives pairs of PST but not MST

Adjacency matrix is $I_n \otimes H(C_4) + \sigma_y \otimes J_{2n}$ for some even positive integer $n$.

In [5]:
def GL_PST_EX(n):
    HC4 = np.matrix([[0,-1j,0,1j],[1j,0,-1j,0],[0,1j,0,-1j],[-1j,0,1j,0]]); HC4 = matrix(HC4)
    SY = np.matrix([[0,-1j],[1j,0]]); SY = matrix(SY)
    Id = identity_matrix(n)
    J = ones_matrix(2*n,2*n)
    return Id.tensor_product(HC4) + SY.tensor_product(J)

In [6]:
n = 4
A = GL_PST_EX(n)
U = transfer(A, np.pi/2).round(5)
U

[0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0]
[0.0 0.0 0

In [7]:
E, eVal = idempotents(A)
for val in eVal:
    print(val.round(5))

-8.0
-2.0
-0.0
2.0
8.0
